In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark: SparkSession = (
    SparkSession
    .builder
    .enableHiveSupport() # type: ignore
    .master('local[*]')
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/02 19:09:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark.sql("SHOW DATABASES").show()

25/04/02 19:09:33 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
25/04/02 19:09:33 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
25/04/02 19:09:39 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
25/04/02 19:09:39 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore sucol@192.168.1.13


+---------+
|namespace|
+---------+
|  default|
|ingestion|
+---------+



In [3]:
spark.sql("SHOW TABLES IN ingestion").show()

25/04/02 19:09:49 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


+---------+----------+-----------+
|namespace| tableName|isTemporary|
+---------+----------+-----------+
|ingestion|     order|      false|
|ingestion|order_item|      false|
|ingestion|   product|      false|
|ingestion|      user|      false|
+---------+----------+-----------+



In [4]:
df_order = spark.read.table("ingestion.order")
df_order.show()

+---+-------+-------------------+---------+-----------+-------------------+-------------------+
| id|user_id|         order_date|   status|total_value|      creation_date| ingestion_datetime|
+---+-------+-------------------+---------+-----------+-------------------+-------------------+
|  1|      2|2024-12-07 18:44:57|  shipped|     1953.0|2025-04-02 18:44:56|2025-04-02 19:07:09|
|  2|      1|2024-08-06 18:44:57|  pending|    5842.57|2025-04-02 18:44:56|2025-04-02 19:07:09|
|  3|      2|2025-01-31 18:44:57| canceled|     2441.8|2025-04-02 18:44:56|2025-04-02 19:07:09|
|  4|      1|2024-08-30 18:44:57| approved|    1208.68|2025-04-02 18:44:56|2025-04-02 19:07:09|
|  5|      2|2024-04-02 18:44:57|delivered|    1583.34|2025-04-02 18:44:56|2025-04-02 19:07:09|
|  6|      1|2024-12-01 18:44:57|  shipped|    7614.99|2025-04-02 18:44:56|2025-04-02 19:07:09|
|  7|      1|2024-09-10 18:44:57|  shipped|    3779.53|2025-04-02 18:44:56|2025-04-02 19:07:09|
|  8|      2|2024-12-07 18:44:57| approv

In [10]:
is_approved = F.col('status') == "approved"
df_approved_order = df_order.filter(is_approved)
df_approved_order.show()

+---+-------+-------------------+--------+-----------+-------------------+--------------------+
| id|user_id|         order_date|  status|total_value|      creation_date|  ingestion_datetime|
+---+-------+-------------------+--------+-----------+-------------------+--------------------+
|  2|      1|2024-12-13 12:27:49|approved|      64.24|2025-03-30 12:27:49|2025-03-31 21:19:...|
| 17|      1|2024-08-17 12:27:49|approved|    2587.44|2025-03-30 12:27:49|2025-03-31 21:19:...|
| 20|      2|2025-03-28 12:27:49|approved|    1960.32|2025-03-30 12:27:49|2025-03-31 21:19:...|
| 21|      1|2024-06-20 12:27:49|approved|    1085.54|2025-03-30 12:27:49|2025-03-31 21:19:...|
| 26|      1|2024-10-29 12:27:49|approved|    1789.42|2025-03-30 12:27:49|2025-03-31 21:19:...|
| 40|      1|2024-10-16 12:27:49|approved|     968.32|2025-03-30 12:27:49|2025-03-31 21:19:...|
| 42|      2|2024-07-23 12:27:49|approved|    3580.78|2025-03-30 12:27:49|2025-03-31 21:19:...|
| 43|      1|2024-12-21 12:27:49|approve

In [13]:
df_user = spark.read.table("ingestion.user")
df_user = df_user.select('id', 'email')

df_approved_order_with_user_email = df_approved_order.join(
    df_user,
    df_approved_order['user_id'] == df_user['id'],
    'left'
)

df_approved_order_with_user_email = df_approved_order_with_user_email.select(
    df_approved_order['*'],
    df_user['email']
)

df_approved_order_with_user_email.show()

+---+-------+-------------------+--------+-----------+-------------------+--------------------+--------------------+
| id|user_id|         order_date|  status|total_value|      creation_date|  ingestion_datetime|               email|
+---+-------+-------------------+--------+-----------+-------------------+--------------------+--------------------+
|  2|      1|2024-12-13 12:27:49|approved|      64.24|2025-03-30 12:27:49|2025-03-31 21:19:...|richardskim@examp...|
| 17|      1|2024-08-17 12:27:49|approved|    2587.44|2025-03-30 12:27:49|2025-03-31 21:19:...|richardskim@examp...|
| 21|      1|2024-06-20 12:27:49|approved|    1085.54|2025-03-30 12:27:49|2025-03-31 21:19:...|richardskim@examp...|
| 26|      1|2024-10-29 12:27:49|approved|    1789.42|2025-03-30 12:27:49|2025-03-31 21:19:...|richardskim@examp...|
| 40|      1|2024-10-16 12:27:49|approved|     968.32|2025-03-30 12:27:49|2025-03-31 21:19:...|richardskim@examp...|
| 43|      1|2024-12-21 12:27:49|approved|    2916.12|2025-03-30

In [ ]:
df_grouped = (
    df_approved_order_with_user_email
    .groupBy('user_id', 'email')
    .agg(
        F.count('*').alias('order_count'),
        F.sum('total_value').alias('total_value')
    )
)


+-------+--------------------+-----------+------------------+
|user_id|               email|order_count|       total_value|
+-------+--------------------+-----------+------------------+
|     12|smithjose@example...|         13|          47176.85|
|      1|richardskim@examp...|        206| 590675.7100000005|
|     13| scott49@example.org|         16|          49640.61|
|      6|victoriariddle@ex...|         27| 79803.23000000001|
|     16|zacharymccoy@exam...|          2|            7696.3|
|      3|kimberlygreen@exa...|         88| 271171.3199999999|
|      5|byrdnicole@exampl...|         64|172332.34999999995|
|     15|kimdonald@example...|          7|          21694.65|
|      9| randy06@example.org|         35|117585.10999999999|
|      4|amanda40@example.net|         70|212784.69000000012|
|      8|thomasethan@examp...|         18| 68314.16999999998|
|      7|kjohnson@example.org|         37|118297.11000000002|
|     10|andreaperez@examp...|         28| 77400.16999999998|
|     11

In [ ]:

# create a column rank based on total_value. The greater the total_value, the higher the rank
df_ranked = df_grouped.selectExpr(
    '*',
    'rank() over (order by total_value desc) as rank'
)

is_top_3 = F.col('rank') <= 3
df_top_3 = df_ranked.filter(is_top_3)

df_top_3.show()

25/03/31 21:36:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/31 21:36:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/31 21:36:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/31 21:36:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/31 21:36:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/31 21:36:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/31 2

+-------+--------------------+-----------+-----------------+----+
|user_id|               email|order_count|      total_value|rank|
+-------+--------------------+-----------+-----------------+----+
|      1|richardskim@examp...|        206|590675.7100000005|   1|
|      2|jimenezjoshua@exa...|        164|528625.2299999997|   2|
|      3|kimberlygreen@exa...|         88|271171.3199999999|   3|
+-------+--------------------+-----------+-----------------+----+



25/03/31 21:36:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/31 21:36:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
